# IoT GNN Demo

## Env Setup
To use this notebook, a conda environment has to be created beforehand. Commands to create the environment and install the minimal packages:
```bash
conda create -n iot_env python=3.11
conda activate iot_env
conda install pytorch
conda install pandas
conda install scikit-learn
conda install matplotlib
conda install pytorch::torchdata
conda install conda-forge::dgl
```  
Then, the conda environment has to be exported.  
```bash
conda install ipykernel
python -m ipykernel install --user --name iot_env --display-name "Python (iot_env)"
```  
Select the iot_env kernel to run the ipynb.

## Imports

In [15]:
from configuration import *

from models import e_graphsage, fnn_model
from data import IoTDataset
from train import ModelTrainer

## Load Data
This assumes there is a "data" folder under the root of the IoT_GNN repo.  
Datasets can be found at: https://drive.google.com/drive/folders/14t41P09gXTsCqPx3YFN1Pruwb2eZQrkT?usp=share_link

In [16]:
multiclass = True
train_data = IoTDataset(version=1, multiclass=multiclass)
val_data = IoTDataset(version=1, multiclass=multiclass, split='val')

## Sanity Check
Running forward pass on the untrained E_GraphSAGE model.

In [3]:
dgl_graph = train_data[0]
temp = e_graphsage.E_GraphSAGE(numLayers=2, num_edge_attr=8, dim_node_embed=128, num_classes=5)
temp.forward(dgl_graph)

Graph(num_nodes=433043, num_edges=420068,
      ndata_schemes={'node_attr': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={'edge_attr': Scheme(shape=(8,), dtype=torch.float32), 'edge_label': Scheme(shape=(), dtype=torch.int64), 'edge_pred': Scheme(shape=(5,), dtype=torch.float32)})

## Model Initialization

In [17]:
model = e_graphsage.E_GraphSAGE(numLayers=2,
                                    dim_node_embed=64,
                                    num_edge_attr=train_data.num_features,
                                    num_classes=len(train_data.classes)
                                    )
model2 = fnn_model.TestFNN(num_hidden_layers=3,
                               hidden_layer_widths=[64, 64, 128],  # Should be approximately comparable to EGS
                               num_edge_attr=train_data.num_features,
                               num_classes=len(train_data.classes),
                               )

In [18]:
training_config = {
        'num_epochs': 100,
        'lr': 1e-3,
        'gpu': False,
        'lr_sched_factor': np.sqrt(10),
        'lr_sched_patience': 10,
    }

## Training

In [19]:
# Instantiate Trainer
trainer = ModelTrainer(training_config, train_data, val_data)

In [ ]:
# Train the GNN
trainer.train_model(model, False)

Training E_GraphSAGE_K2: 100%|██████████| 100/100 [00:58<00:00,  1.71epoch/s, train loss=0.8619, validation loss=0.8926, learning rate=1.00e-03, F1 score=0.76706]


In [ ]:
# Train the FCNN
trainer.train_model(model2, False)

Training FCNN_K3: 100%|██████████| 100/100 [00:54<00:00,  1.83epoch/s, train loss=1.1981, validation loss=0.8992, learning rate=1.00e-03, F1 score=0.77914]
